# STARE-based Integrative Analysis of Diverse Data Using Dask Parallel Programming Demo Paper

In [1]:
print('hello world')

hello world


## Preliminaries

## STARE HTM Geolocation

## STARE HTM Trixel Neighborhoods

## STARE Cover of a Granule

## STARE Region of Interest

## STARE Sidecar Files and Cataloguing

## Searching the Catalogue

## Subsetting the Data

## Dask Acceleration

### Dask Preliminaries

### Example Parallelization

### Trixel-based Integration

## Conclusion